## 🛠️ Installation and set-up

In [ ]:
!pip install --upgrade pip
!pip install "jax[tpu]>=0.2.16" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

In [ ]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()
# import jax
jax.local_devices()

In [ ]:
# Install required libraries
!pip install -q dalle-mini
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git

In [12]:
!rm -rf /content/train_vid_aa_ac
!rm -rf /content/sorted_train_vid_aa_ac.tsv

## Create TSV

In [13]:
#@title classes
all_classes_map = {"clay pottery making": 0, "testifying": 1, "news anchoring": 2, "using bagging machine": 3, "climbing a rope": 4, "tickling": 5, "watering plants": 6, "eating spaghetti": 7, "dribbling basketball": 8, "calligraphy": 9, "playing tennis": 10, "chiseling stone": 11, "crossing eyes": 12, "mountain climber (exercise)": 13, "tap dancing": 14, "planing wood": 15, "playing pinball": 16, "delivering mail": 17, "riding a bike": 18, "putting in contact lenses": 19, "shaving head": 20, "doing jigsaw puzzle": 21, "playing with trains": 22, "wading through water": 23, "brushing teeth": 24, "washing hair": 25, "opening bottle (not wine)": 26, "abseiling": 27, "making balloon shapes": 28, "inflating balloons": 29, "pouring beer": 30, "preparing salad": 31, "feeding birds": 32, "cutting watermelon": 33, "petting cat": 34, "doing aerobics": 35, "sausage making": 36, "throwing knife": 37, "head stand": 38, "dining": 39, "throwing water balloon": 40, "opening present": 41, "chopping meat": 42, "making paper aeroplanes": 43, "folding clothes": 44, "javelin throw": 45, "changing gear in car": 46, "pushing wheelchair": 47, "vacuuming floor": 48, "shuffling feet": 49, "recording music": 50, "historical reenactment": 51, "playing drums": 52, "kicking field goal": 53, "snowkiting": 54, "smelling feet": 55, "lighting fire": 56, "curling (sport)": 57, "eating cake": 58, "bee keeping": 59, "cutting apple": 60, "drooling": 61, "gymnastics tumbling": 62, "archaeological excavation": 63, "clapping": 64, "visiting the zoo": 65, "marriage proposal": 66, "petting animal (not cat)": 67, "egg hunting": 68, "planting trees": 69, "biking through snow": 70, "playing ping pong": 71, "cutting nails": 72, "battle rope training": 73, "playing scrabble": 74, "playing gong": 75, "lock picking": 76, "poking bellybutton": 77, "shearing sheep": 78, "photocopying": 79, "sword fighting": 80, "making snowman": 81, "changing wheel (not on bike)": 82, "playing kickball": 83, "threading needle": 84, "somersaulting": 85, "playing harmonica": 86, "passing american football (not in game)": 87, "shaving legs": 88, "sign language interpreting": 89, "opening wine bottle": 90, "person collecting garbage": 91, "massaging neck": 92, "hurling (sport)": 93, "using a power drill": 94, "golf chipping": 95, "using a microscope": 96, "driving tractor": 97, "yawning": 98, "huddling": 99, "bandaging": 100, "tapping pen": 101, "playing rubiks cube": 102, "playing flute": 103, "canoeing or kayaking": 104, "training dog": 105, "cracking knuckles": 106, "roasting marshmallows": 107, "hula hooping": 108, "dancing ballet": 109, "playing paintball": 110, "waxing eyebrows": 111, "stretching arm": 112, "falling off bike": 113, "snowboarding": 114, "base jumping": 115, "skipping rope": 116, "getting a haircut": 117, "assembling bicycle": 118, "windsurfing": 119, "golf putting": 120, "fixing bicycle": 121, "riding camel": 122, "zumba": 123, "playing trumpet": 124, "eating watermelon": 125, "shooting basketball": 126, "blowing nose": 127, "embroidering": 128, "grooming horse": 129, "leatherworking": 130, "motorcycling": 131, "hugging baby": 132, "unboxing": 133, "dyeing eyebrows": 134, "getting a tattoo": 135, "marching": 136, "swinging on something": 137, "assembling computer": 138, "tying knot (not on a tie)": 139, "sanding floor": 140, "opening door": 141, "tying shoe laces": 142, "jumping bicycle": 143, "skateboarding": 144, "parasailing": 145, "winking": 146, "eating doughnuts": 147, "shooting goal (soccer)": 148, "fly tying": 149, "ice fishing": 150, "longboarding": 151, "flying kite": 152, "making a cake": 153, "putting on eyeliner": 154, "snowmobiling": 155, "twiddling fingers": 156, "changing oil": 157, "cutting orange": 158, "tagging graffiti": 159, "using circular saw": 160, "chiseling wood": 161, "shucking oysters": 162, "baking cookies": 163, "playing poker": 164, "making sushi": 165, "waking up": 166, "setting table": 167, "giving or receiving award": 168, "making a sandwich": 169, "pole vault": 170, "polishing metal": 171, "shoveling snow": 172, "climbing ladder": 173, "playing polo": 174, "sawing wood": 175, "whistling": 176, "alligator wrestling": 177, "juggling soccer ball": 178, "eating hotdog": 179, "passing American football (in game)": 180, "watching tv": 181, "playing organ": 182, "hopscotch": 183, "skydiving": 184, "dancing charleston": 185, "roasting pig": 186, "geocaching": 187, "ironing": 188, "putting on mascara": 189, "kicking soccer ball": 190, "massaging feet": 191, "trapezing": 192, "dodgeball": 193, "waxing legs": 194, "drumming fingers": 195, "licking": 196, "luge": 197, "laying concrete": 198, "eating ice cream": 199, "burping": 200, "archery": 201, "reading newspaper": 202, "breakdancing": 203, "making tea": 204, "hurdling": 205, "cracking back": 206, "cooking scallops": 207, "shaping bread dough": 208, "writing": 209, "knitting": 210, "shuffling cards": 211, "blowing bubble gum": 212, "playing volleyball": 213, "tasting food": 214, "welding": 215, "breathing fire": 216, "singing": 217, "mowing lawn": 218, "ice climbing": 219, "chopping vegetables": 220, "picking fruit": 221, "building shed": 222, "sharpening knives": 223, "blasting sand": 224, "lifting hat": 225, "building lego": 226, "hitting baseball": 227, "tapping guitar": 228, "cartwheeling": 229, "golf driving": 230, "carving ice": 231, "casting fishing line": 232, "bending metal": 233, "punching bag": 234, "sleeping": 235, "tackling": 236, "riding scooter": 237, "sled dog racing": 238, "karaoke": 239, "swimming butterfly stroke": 240, "playing beer pong": 241, "using segway": 242, "moon walking": 243, "fencing (sport)": 244, "playing ice hockey": 245, "chopping wood": 246, "hugging (not baby)": 247, "throwing tantrum": 248, "auctioning": 249, "drawing": 250, "drop kicking": 251, "playing saxophone": 252, "busking": 253, "docking boat": 254, "waxing back": 255, "sticking tongue out": 256, "trimming shrubs": 257, "lunge": 258, "catching or throwing softball": 259, "sweeping floor": 260, "swing dancing": 261, "chewing gum": 262, "air drumming": 263, "playing ukulele": 264, "playing controller": 265, "playing blackjack": 266, "attending conference": 267, "cosplaying": 268, "raising eyebrows": 269, "sneezing": 270, "grinding meat": 271, "skiing slalom": 272, "peeling potatoes": 273, "gold panning": 274, "milking cow": 275, "springboard diving": 276, "pirouetting": 277, "high jump": 278, "celebrating": 279, "packing": 280, "cleaning windows": 281, "calculating": 282, "situp": 283, "riding mule": 284, "tossing coin": 285, "playing laser tag": 286, "weaving basket": 287, "using remote controller (not gaming)": 288, "popping balloons": 289, "kissing": 290, "ice skating": 291, "jumpstyle dancing": 292, "contact juggling": 293, "standing on hands": 294, "jumping into pool": 295, "photobombing": 296, "jumping jacks": 297, "putting on lipstick": 298, "smoking": 299, "shaking head": 300, "playing field hockey": 301, "shining flashlight": 302, "extinguishing fire": 303, "card stacking": 304, "breaking boards": 305, "riding mechanical bull": 306, "headbanging": 307, "playing monopoly": 308, "skipping stone": 309, "driving car": 310, "playing dominoes": 311, "deadlifting": 312, "juggling fire": 313, "squat": 314, "applying cream": 315, "making jewelry": 316, "playing harp": 317, "push up": 318, "scrapbooking": 319, "mushroom foraging": 320, "throwing snowballs": 321, "riding unicycle": 322, "playing accordion": 323, "bull fighting": 324, "shot put": 325, "kitesurfing": 326, "crawling baby": 327, "riding snow blower": 328, "cheerleading": 329, "pillow fight": 330, "robot dancing": 331, "country line dancing": 332, "spray painting": 333, "finger snapping": 334, "arm wrestling": 335, "skiing mono": 336, "bartending": 337, "roller skating": 338, "skiing crosscountry": 339, "playing bass guitar": 340, "sipping cup": 341, "climbing tree": 342, "riding elephant": 343, "yoga": 344, "contorting": 345, "tai chi": 346, "trimming trees": 347, "clam digging": 348, "pull ups": 349, "bulldozing": 350, "playing basketball": 351, "shaking hands": 352, "catching fish": 353, "playing netball": 354, "pushing wheelbarrow": 355, "blowdrying hair": 356, "separating eggs": 357, "tying necktie": 358, "using puppets": 359, "playing bagpipes": 360, "blowing out candles": 361, "breading or breadcrumbing": 362, "acting in play": 363, "falling off chair": 364, "fixing hair": 365, "walking the dog": 366, "laying stone": 367, "playing maracas": 368, "arranging flowers": 369, "doing nails": 370, "disc golfing": 371, "front raises": 372, "bouncing on bouncy castle": 373, "bending back": 374, "capoeira": 375, "plastering": 376, "bookbinding": 377, "braiding hair": 378, "sword swallowing": 379, "using a sledge hammer": 380, "putting on shoes": 381, "presenting weather forecast": 382, "jaywalking": 383, "wood burning (art)": 384, "brush painting": 385, "washing feet": 386, "building cabinet": 387, "pinching": 388, "tie dying": 389, "playing recorder": 390, "using atm": 391, "spinning poi": 392, "rock scissors paper": 393, "wrapping present": 394, "eating chips": 395, "walking through snow": 396, "looking at phone": 397, "making bubbles": 398, "pushing car": 399, "talking on cell phone": 400, "capsizing": 401, "hammer throw": 402, "waving hand": 403, "steer roping": 404, "smashing": 405, "faceplanting": 406, "fidgeting": 407, "playing pan pipes": 408, "spelunking": 409, "catching or throwing baseball": 410, "cleaning shoes": 411, "punching person (boxing)": 412, "rope pushdown": 413, "making pizza": 414, "feeding fish": 415, "cutting pineapple": 416, "frying vegetables": 417, "adjusting glasses": 418, "ironing hair": 419, "playing guitar": 420, "wrestling": 421, "dancing macarena": 422, "playing lute": 423, "lawn mower racing": 424, "building sandcastle": 425, "mosh pit dancing": 426, "playing trombone": 427, "playing didgeridoo": 428, "sailing": 429, "pumping fist": 430, "land sailing": 431, "dumpster diving": 432, "cumbia": 433, "snorkeling": 434, "clean and jerk": 435, "water skiing": 436, "dancing gangnam style": 437, "carrying baby": 438, "rock climbing": 439, "bottling": 440, "folding napkins": 441, "parkour": 442, "using a wrench": 443, "flipping pancake": 444, "checking tires": 445, "bouncing on trampoline": 446, "decorating the christmas tree": 447, "krumping": 448, "coloring in": 449, "tobogganing": 450, "weaving fabric": 451, "tiptoeing": 452, "playing cello": 453, "trimming or shaving beard": 454, "playing ocarina": 455, "scuba diving": 456, "swimming backstroke": 457, "throwing axe": 458, "scrubbing face": 459, "repairing puncture": 460, "wading through mud": 461, "getting a piercing": 462, "long jump": 463, "snatch weight lifting": 464, "card throwing": 465, "playing darts": 466, "tying bow tie": 467, "cleaning pool": 468, "laughing": 469, "tango dancing": 470, "bowling": 471, "surfing crowd": 472, "playing squash or racquetball": 473, "shining shoes": 474, "opening refrigerator": 475, "hoverboarding": 476, "diving cliff": 477, "triple jump": 478, "playing xylophone": 479, "hockey stop": 480, "sharpening pencil": 481, "putting on foundation": 482, "swinging baseball bat": 483, "barbequing": 484, "slapping": 485, "bodysurfing": 486, "exercising with an exercise ball": 487, "swimming breast stroke": 488, "waxing chest": 489, "salsa dancing": 490, "ice swimming": 491, "catching or throwing frisbee": 492, "riding or walking with horse": 493, "feeding goats": 494, "unloading truck": 495, "throwing discus": 496, "washing hands": 497, "playing chess": 498, "massaging back": 499, "playing piano": 500, "playing cymbals": 501, "cooking sausages (not on barbeque)": 502, "texting": 503, "smoking pipe": 504, "curling hair": 505, "playing violin": 506, "dunking basketball": 507, "cleaning gutters": 508, "making horseshoes": 509, "backflip (human)": 510, "square dancing": 511, "slacklining": 512, "bench pressing": 513, "using a paint roller": 514, "directing traffic": 515, "crying": 516, "laying tiles": 517, "gospel singing in church": 518, "reading book": 519, "high kick": 520, "mopping floor": 521, "combing hair": 522, "applauding": 523, "holding snake": 524, "playing marbles": 525, "cracking neck": 526, "putting on sari": 527, "counting money": 528, "installing carpet": 529, "laying bricks": 530, "cooking egg": 531, "waiting in line": 532, "carving pumpkin": 533, "making the bed": 534, "passing soccer ball": 535, "bungee jumping": 536, "smoking hookah": 537, "cooking on campfire": 538, "yarn spinning": 539, "shopping": 540, "flint knapping": 541, "playing badminton": 542, "eating carrots": 543, "blowing glass": 544, "headbutting": 545, "pushing cart": 546, "answering questions": 547, "swimming front crawl": 548, "playing clarinet": 549, "running on treadmill": 550, "tasting beer": 551, "belly dancing": 552, "home roasting coffee": 553, "eating burger": 554, "cleaning toilet": 555, "brushing hair": 556, "stomping grapes": 557, "grooming dog": 558, "making cheese": 559, "peeling apples": 560, "arguing": 561, "scrambling eggs": 562, "jogging": 563, "beatboxing": 564, "surfing water": 565, "water sliding": 566, "side kick": 567, "throwing ball (not baseball or American football)": 568, "hand washing clothes": 569, "needle felting": 570, "dyeing hair": 571, "using inhaler": 572, "paragliding": 573, "playing hand clapping games": 574, "jetskiing": 575, "ski jumping": 576, "playing keyboard": 577, "ripping paper": 578, "massaging legs": 579, "drinking shots": 580, "pumping gas": 581, "massaging person's head": 582, "rolling pastry": 583, "folding paper": 584, "blowing leaves": 585, "bathing dog": 586, "sewing": 587, "staring": 588, "washing dishes": 589, "tasting wine": 590, "playing cricket": 591, "sucking lolly": 592, "moving furniture": 593, "juggling balls": 594, "stretching leg": 595, "doing laundry": 596, "bobsledding": 597, "crossing river": 598, "tightrope walking": 599, "tmp": 600}

In [14]:
!unzip -q '/content/drive/MyDrive/dataset_train_zip/train_vid_ab_ab.zip' -d '/content/'

In [15]:
import os
import csv
import pandas as pd
import numpy as np
import sys
def main(out_file_path):
  with open(out_file_path, 'wt', encoding='utf-8') as f:  
    tsv_writer = csv.writer(f, delimiter='\t') 
    # tsv_writer.writerow(['fullPath', 'latent'])
    for categories, subdirs, files in os.walk("/content/train_vid_ab_ab"): # "/content/drive/MyDrive/train_vid" 
          # print(subdir)  
          for subdir in subdirs:
            if "frames_" in subdir:
              # print(subdir)
              p = os.path.join(categories, subdir)
              listOfFile = os.listdir(p)              
              # print(listOfFile)
              fullPathList = []
              # print(len(listOfFile))
              for entry in listOfFile:                         
                fullPath = os.path.join(p, entry)
                fullPathList.append(fullPath)
                # print('full path: ', fullPath)
                # print()
                # print("key sort: ", (fullPath.replace(p, '').replace('.jpg','')).split("_")[2])
                fullPathList.sort(key= lambda x: float((x.replace(p, '').replace('.jpg','')).split("_")[2]))
                # latent = np.random.rand(24,24, 256)
              for i in range(len(fullPathList)):
                path = fullPathList[i]
                tsv_writer.writerow([path])
              # sys.exit(0)

out_file_path = '/content/sorted_train_vid_ab_ab.tsv'
main(out_file_path)


In [16]:
!cat /content/*.tsv | wc -l

144672


# Parallel / batches


In [10]:
# VQGAN-JAX - pmap encoding HowTo
import numpy as np
import pandas as pd
# For data loading
import torch
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.folder import default_loader
from torchvision.transforms import InterpolationMode
import torchvision.transforms as T

# For data saving
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from numpy import savetxt
import jax
import json
from jax import pmap
import os
import sys
from vqgan_jax.modeling_flax_vqgan import VQModel

# Load VQGAN model
VQGAN_REPO = "dalle-mini/vqgan_imagenet_f16_16384"
VQGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"
model = VQModel.from_pretrained(
    VQGAN_REPO, revision=VQGAN_COMMIT_ID
)

In [ ]:

## Params and arguments

# List of paths containing images to encode
image_list = '/content/sorted_train_vid_ab_ab.tsv'
output_tsv = '/content/drive/MyDrive/vqgan_out_train_vid_ab_ab.tsv'                # Encoded results
batch_size = 16
num_workers = 4   # TPU v3-8s have 96 cores, so feel free to increase this number when necessary
# original_data = '/content/drive/MyDrive/kinetics600/train.csv'


# model = VQModel.from_pretrained("flax-community/vqgan_f16_16384")

## Data Loading.

# Simple torch Dataset to load images from paths.
# You can use your own pipeline instead.
class ImageDataset(Dataset):
    def __init__(self, image_list_path: str, image_size: int, max_items=None):
        """
        :param image_list_path: Path to a file containing a list of all images. We assume absolute paths for now.
        :param image_size: Image size. Source images will be resized and center-cropped.
        :max_items: Limit dataset size for debugging
        """
        self.image_list = pd.read_csv(image_list_path, sep='\t', header=None)
        if max_items is not None: self.image_list = self.image_list[:max_items]
        self.image_size = image_size
        
    def __len__(self):
        return len(self.image_list)
    
    def _get_raw_image(self, i):

        image_path = Path(self.image_list.iloc[i][0])
        return default_loader(image_path)

    def preprocess_img(self,img, target_image_size=(384,384)): 
      s = min(img.size)
      r = self.image_size / s
      s = (round(r * img.size[1]), round(r * img.size[0]))
      img = TF.resize(img, target_image_size, interpolation=Image.LANCZOS)
      img = TF.center_crop(img, output_size=2 * [target_image_size])
      img = torch.unsqueeze(T.ToTensor()(img), 0)
      return img.permute(0, 2, 3, 1).numpy()

    def resize_image(self, image):
        s = min(image.size)
        r = self.image_size / s
        s = (round(r * image.size[1]), round(r * image.size[0]))
        image = TF.resize(image, s, interpolation=InterpolationMode.LANCZOS)
        image = TF.center_crop(image, output_size = 2 * [self.image_size])
        image = torch.unsqueeze(T.ToTensor()(image), 0)
        return image.permute(0, 2, 3, 1).numpy()
    
    
    def __getitem__(self, i):
        image = self._get_raw_image(i)
        return self.resize_image(image)

## Encoding

# Encoding function to be parallelized with `pmap`
# Note: images have to be square
def encode(model, batch):
    centr, indices = model.encode(batch)
    return centr

# Alternative: create a batch with num_tpus*batch_size and use `shard` to distribute.
def superbatch_generator(dataloader, num_tpus):
    iter_loader = iter(dataloader)
    for batch in iter_loader:
        superbatch = [batch.squeeze(1)]
        try:
            for _ in range(num_tpus-1):
                batch = next(iter_loader)
                if batch is None:
                    break
                # Skip incomplete last batch
                if batch.shape[0] == dataloader.batch_size:
                    superbatch.append(batch.squeeze(1))
        except StopIteration:
            pass
        superbatch = torch.stack(superbatch, axis=0)
        yield superbatch

def find_class(path, class_dict):
  # print(path,type(path))
  path = str(path)
  if ('/' in path):
    split_path = path.split('/')
  if ("\\" in path):
    split_path = path.split('\\')
    
  for i in range(len(split_path)):
    t = split_path[i]
    if (t.startswith('train_vid_a') or t.startswith('test') or t.startswith('valid') or t.startswith('train_vid')) and (i+1 < len(split_path)):
      class_text = split_path[i+1]
      class_no = class_dict[class_text]
      return [class_no]


def create_dic_mapping(file):

  links_df = pd.read_csv(file)
  folders_names = links_df['label'].unique().tolist() + ['tmp']
  class_dict = {}
  for i in range(len(folders_names)):
    class_name = folders_names[i]
    class_dict[class_name] = i

  with open('json_data_dict.json', 'w') as outfile:
    json.dump(class_dict, outfile)
  return class_dict

def encode_dataset(dataset, batch_size=16):
    dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)
    superbatches = superbatch_generator(dataloader, num_tpus=jax.device_count())
    class_dict = all_classes_map #create_dic_mapping(original_file)
    num_tpus = jax.device_count()
    # dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)
    # superbatches = superbatch_generator(dataloader, num_tpus=num_tpus)
    
    p_encoder = pmap(lambda batch: encode(model, batch))

    
    # Save each superbatch to avoid reallocation of buffers as we process them.
    # Keep the file open to prevent excessive file seeks.
    count = 0
    # with open(output_tsv, "w") as file:
    iterations = len(dataset) // (batch_size * num_tpus)
    for n in tqdm(range(iterations)):
        count+=1
        superbatch = next(superbatches)
        encoded = p_encoder(superbatch.numpy())
        
        # Extract paths from the dataset, save paths and encodings (as string)
        start_index = n * batch_size * num_tpus
        end_index = (n+1) * batch_size * num_tpus
        paths = dataset.image_list[start_index:end_index][0].values
        # print(paths.shape)
        # print(paths)
        # encoded_as_string = list(map(lambda item: np.array2string(item, separator=',', max_line_width=500000, formatter={'int':lambda x: str(x)}), encoded))
        batch_df = pd.DataFrame.from_dict({"image_file": paths})
        for i in range(num_tpus):          
          spl = paths[i*16 +1].split('/')
          video_name =  spl[len(spl) -2 ]
           
          class_no = find_class(paths[i*16 +1], class_dict)
          class_no_np = np.array(class_no)     
          # print(encoded[i,:,:,:].shape)
          jax.numpy.savez(os.path.join('/content/drive/MyDrive/dataset_npz/ab_ab',"out_"+str(count) + "_" + video_name), encoded[i,:,:,:], class_no_np)       
        
            
dataset = ImageDataset(image_list, image_size=384)
encoded_dataset = encode_dataset(dataset=dataset, batch_size=batch_size)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 35%|███▌      | 398/1130 [28:32<50:28,  4.14s/it]

In [ ]:
import numpy as np
import jax 
import jax.numpy as jnp

t = jnp.array(np.random.rand(2,3,3,3))
#for i in range device count 
# np.save +16 for the label (i*16)
for i in range(t.shape[0]):
  np.save('out_'+str(i),t[i,:,:,:])

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
